In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def costFun(theta, X_train, Y_train):
    m = X_train.shape[0]
    pred = X_train@theta
    J = np.linalg.norm(pred - Y_train)**2 / (2*m)
    return J

In [ ]:
def diffCostFun(X_train, Y_train, theta):
  m = X_train.shape[0]
  return (1/m) * (((X_train.T @ X_train) @ theta) - (X_train.T @ Y_train)) 
  

In [ ]:
def fitGD(X_train, Y_train, alpha, l, reg, it):
  m = X_train.shape[0]
  n = X_train.shape[1] - 1 
  theta = np.zeros((n+1, 1))
  grad_abs_theta = np.ones((n+1, 1))
  L1 = []
  L2 = []
  L3 = []
  
  for i in range(it):
    L1.append(costFun(theta,X_train,Y_train) + (l/n)*(np.linalg.norm(theta,1)))
    grad_abs_theta[theta < 0] = -1
    grad_L1 = diffCostFun(X_train, Y_train, theta) + (l/n) * (grad_abs_theta)
    
    L2.append(costFun(theta,X_train,Y_train) + (l/n)*(np.linalg.norm(theta,2))) 
    grad_L2 = diffCostFun(X_train, Y_train, theta) + (l/n) * (theta)

    L3.append((L1[i] + L2[i])/2)  #elastic net with equal weight for L1 and L2 (mixing coefficient = 0.5)
    grad_L3 = (grad_L1 + grad_L2) / 2
    
    if reg == 1:
      theta = theta - (alpha * grad_L1)
    elif reg == 2:
      theta = theta - (alpha * grad_L2)
    elif reg == 3:
      theta = theta - (alpha * grad_L3)

  iter = [i for i in range(1,it+1)]
  if(reg==1):
    plt.gca()
    plt.plot(iter, L1, label='Gradient Descent with L1 Regularisation')
    plt.xlabel('Iterations')
    plt.ylabel('Cost Function')
    plt.title('Variation of Cost Function with Iterations')
    plt.legend()
    plt.show
  if(reg==2):
    plt.gca()
    plt.plot(iter,L2, label='Gradient Descent with L2 Regularisation')
    plt.xlabel('Iterations')
    plt.ylabel('Cost Function')
    plt.title('Variation of Cost Function with Iterations')
    plt.legend()
    plt.show
  if(reg==3):
    plt.gca()
    plt.plot(iter,L3label='Gradient Descent with Elastic Net Regularisation')  
    plt.xlabel('Iterations')
    plt.ylabel('Cost Function')
    plt.title('Variation of Cost Function with Iterations')
    plt.legend()
    plt.show

  return theta

In [ ]:
def fitNormal(X_train,Y_train):
  theta = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ Y_train
  return theta

In [ ]:
def locallyWeighted(X_train, Y_train, x, tau, alpha, it):
  m = X_train.shape[0]
  n = X_train.shape[1] - 1 
  
  pred = np.zeros((x.shape[0]))
  pred_ = np.zeros((x.shape[0]))
  
  for i in range(x.shape[0]):
    point = x[i]
    w = np.zeros((m,m))
    theta = np.zeros((n+1,1))
    for j in range(m):
      temp = -(X_train[j]-point) @ (X_train[j]-point).T
      w[j][j] = np.exp(temp/(2 * (tau**2)))
    for k in range(it):
      # J = (1/(2*m)) * (((X_train@theta) - Y_train).T )@w@(X_train@theta - Y_train)
      grad_J = (1/m) * (X_train.T @ w @ ((X_train @ theta) - Y_train))
      theta = theta - (alpha* grad_J)
    pred[i] = point @ theta

  return pred